In [1]:

import argparse
import os
import numpy as np
import pandas as pd
from nsd_access import NSDAccess
import scipy.io

from config import NSD_ROOT_DIR, DATA_ROOT_DIR
from tqdm import tqdm


subj = 'subj01'
atlasname = 'streams'

proxy_list = ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']
for proxy in proxy_list:
    os.environ[proxy] = 'http://58.34.83.134:31280'

nsda = NSDAccess(NSD_ROOT_DIR)
nsd_expdesign = scipy.io.loadmat(os.path.join(NSD_ROOT_DIR, 'nsddata/experiments/nsd/nsd_expdesign.mat'))

# for downlaod annotations
nsda.read_image_coco_info([0])

downloading annotations from http://images.cocodataset.org/annotations/annotations_trainval2017.zip
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


[{'image_id': 532481,
  'id': 541125,
  'caption': "A person kitesurfing over the waves of the ocean's shore."},
 {'image_id': 532481,
  'id': 542259,
  'caption': 'a kite surfer is doing a flying trick over some water'},
 {'image_id': 532481,
  'id': 547713,
  'caption': 'A man is flying up in the air and having fun. '},
 {'image_id': 532481,
  'id': 554427,
  'caption': 'A guy is waterboarding in the ocean on a windy day.'},
 {'image_id': 532481,
  'id': 558036,
  'caption': 'A person kite boarding in rough seas near the shoreline.'}]

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

size = 425
def calc(x, len, h, l, r):
    if (len <= 0):
        return 0
    l = h * l
    r = h * (1 - r)
    y = x + len
    return max(0, (min(y, r) - max(x, l)) / len)

def resize_bbox(bbox, cropBox, h, w):
    ratio = calc(bbox[0], bbox[2], w, cropBox[2], cropBox[3]) * calc(bbox[1], bbox[3], h, cropBox[0], cropBox[1])

    bbox[1] -= h * cropBox[0]
    h *= (1 - cropBox[0] - cropBox[1])

    bbox[0] -= w * cropBox[2]
    w = w * (1 - cropBox[2] - cropBox[3])

    bbox[0] *= size / w
    bbox[2] *= size / w

    bbox[1] *= size / h
    bbox[3] *= size / h

    # Boundary processing
    bbox[0] = max(0, bbox[0])
    bbox[1] = max(0, bbox[1])
    bbox[2] = min(size - bbox[0], bbox[2])
    bbox[3] = min(size - bbox[1], bbox[3])

    sratio = (bbox[2] * bbox[3]) / (size * size)

    return bbox, ratio, sratio


In [3]:

class_labels = [i for i in range(0, 200)]
for x in nsda.get_coco_cats_list():
    class_labels[x['id']] = x['name']

print(f"In total {len(class_labels)}")

loading annotations into memory...
Done (t=0.41s)
creating index...
index created!
In total 200


In [4]:

st = 0
en = 73000

id_list = list(range(st, en))
info_list = nsda.get_image_info(id_list)
info_Img_list = nsda.read_image_coco_info_Img(id_list, info_type = 'instances')
annotations_list = nsda.read_image_coco_info(id_list, info_type='instances')


loading annotations into memory...
Done (t=10.19s)
creating index...
index created!
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!
loading annotations into memory...
Done (t=12.09s)
creating index...
index created!
loading annotations into memory...
Done (t=1.81s)
creating index...
index created!


In [5]:
n = 73000
show = False
filter_ratio = 0

In [6]:
st = 0
en = st + n

id_list = id_list[st:en]
info_list = info_list[st:en]
info_Img_list = info_Img_list[st:en]
annotations_list = annotations_list[st:en]

outputs = nsda.get_coco_dataset().copy()
outputs['images'] = []
outputs['annotations'] = []
tot_annotation = 0


for i, (info, info_Img, annotations) in enumerate(tqdm(list(zip(info_list, info_Img_list, annotations_list)))):
    info_Img_new = info_Img[0].copy()
    info_Img_new['id'] = i
    info_Img_new['width'] = size
    info_Img_new['height'] = size
    info_Img_new['file_name'] = f'{i:06}.png'
    outputs['images'].append(info_Img_new)

    cropBox = info['cropBox']
    cropBox = eval(cropBox)
    h, w = info_Img[0]['height'], info_Img[0]['width']

    if (show):
        img = nsda.read_images(i, show=False)
        # Draw the image
        fig, ax = plt.subplots()
        ax.imshow(img)

    # Draw the annotations with class labels
    for annotation in annotations:
        # print(annotation)
        bbox = annotation['bbox'].copy()
        bbox, ratio, sratio = resize_bbox(bbox, cropBox, h, w)

        if ratio < 0.5:
            continue

        if sratio < filter_ratio:
            continue
        # print(sratio)

        annotation_new = annotation.copy()
        annotation_new['image_id'] = i
        annotation_new['bbox'] = bbox
        annotation_new['id'] = tot_annotation
        tot_annotation += 1
        outputs['annotations'].append(annotation_new)



        if (show):
            category_id = annotation['category_id']
            class_label = class_labels[category_id]  # Adjust index to match class_labels list
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(bbox[0], bbox[1], class_label, color='r')

    if (show):
        # Show the image with annotations
        plt.show()

import json

outputs_path = os.path.join(DATA_ROOT_DIR, f'instances_{st}_{en}_{filter_ratio}.json')
with open(outputs_path, "w") as f:
    json.dump(outputs, f)
    

loading annotations into memory...
Done (t=0.36s)
creating index...
index created!


100%|██████████| 73000/73000 [00:03<00:00, 19387.50it/s]
